In [1]:
%run document_generator.ipynb

In [2]:
import os
from dataclasses import dataclass
from random import randint
from typing import Iterable, Callable, Tuple

import numpy as np
import sympy as sp
from matplotlib import pyplot as plt
from numpy.random import choice
from sympy import latex
import matplotlib

In [3]:
matplotlib.use('Agg')

In [4]:
def latex_image(path: str) -> str:
    return r"""\begin{figure}[H]
    \centering
    \includegraphics[width=0.5\textwidth]{%s}
    \label{fig:figure}
\end{figure}""" % path

In [5]:
@dataclass
class MyFunc:
    sympy_version: Callable[[sp.Expr], sp.Expr]
    numpy_version: Callable[[np.array], np.array]


def random_float_coef():
    return sp.S(choice(list(set(range(-3, 4)) - {0}))) / choice([1, 2, 3])


def simple_func_generator(sympy_version: Callable[[sp.Expr], sp.Expr],
                          numpy_version: Callable[[np.array], np.array]) -> Callable[[], MyFunc]:
    def generate():
        coef1 = random_float_coef()
        coef2 = random_float_coef()
        return MyFunc(
            lambda symbol: coef1 * sympy_version(coef2 * symbol),
            lambda arr: float(coef1) * numpy_version(float(coef2) * arr)
        )

    return generate


generate_sin = simple_func_generator(sp.sin, np.sin)
generate_cos = simple_func_generator(sp.cos, np.cos)
generate_exp = simple_func_generator(sp.exp, np.exp)


def generate_polynom() -> MyFunc:
    coefs = np.array([sp.S(choice(list(range(-3, 4)) + i * [0])) / choice([1, 2, 3]) for i in range(4)])
    float_coefs = list(map(float, coefs))
    return MyFunc(
        lambda symbol: (coefs * np.array([symbol ** i for i in range(4)])).sum(),
        lambda arr: float_coefs[0] +
                    float_coefs[1] * arr +
                    float_coefs[2] * arr ** 2 +
                    float_coefs[3] * arr ** 3
    )


def generate_raising_arg_to_deg() -> MyFunc:
    coef1 = random_float_coef()
    coef2 = sp.S(choice([2, 3, 5])) / choice([1, 2, 4])
    if coef2 == 1:
        coef2 = 2
    return MyFunc(
        lambda symbol: coef1 * coef2 ** symbol,
        lambda arr: float(coef1) * float(coef2) ** arr
    )

In [6]:
def add(x, y):
    return lambda arg: x(arg) + y(arg)


def mul(x, y):
    return lambda arg: x(arg) * y(arg)


def aggregate(x, y):
    return lambda arg: x(y(arg))


def generate_func() -> MyFunc:
    operators = [add, mul, aggregate]
    generators = [generate_cos, generate_sin, generate_exp, generate_polynom, generate_raising_arg_to_deg]
    func1, func2 = map(lambda gen: gen(), choice(generators, 2, replace=False))
    operator = choice(operators)
    return MyFunc(
        operator(func1.sympy_version, func2.sympy_version),
        operator(func1.numpy_version, func2.numpy_version)
    )

In [7]:
DESCRIPTION = "Построить в одной координатной плоскости графики функций."

OUT_ANSWER_PATH = "../out/answers/answer4/"

In [8]:
def task_condition(func1: sp.Expr,
                   func2: sp.Expr,
                   left_border: sp.core.numbers.Number, right_border: sp.core.numbers.Number,
                   color1: str, line_type1: str,
                   color2: str, line_type2: str,
                   mark_step: sp.core.numbers.Number,
                   vertical_marks: Iterable[int]):
    return '\n\n' + rf"Построить в одной координатной плоскости графики функций $$f(x) = {latex(func1)}$$ $$g(x) = {latex(func2)}$$ на отрезке $\left[{latex(left_border)}, \  {latex(right_border)}\right]$, $f(x)$ {color1} {line_type1}, $g(x)$ {color2} {line_type2}. Отметки на горизонтальной оси от ${latex(left_border)}$ до ${latex(right_border)}$ с шагом ${latex(mark_step)}$, отметки подписать формулами как в Примере 3. По вертикальной оси отметки ${'$, $'.join(map(str, vertical_marks))}$."

In [28]:
def solve_task(option: int,
               func1: Callable[[np.array], np.array],
               func2: Callable[[np.array], np.array],
               left_border: sp.core.numbers.Number, right_border: sp.core.numbers.Number,
               color1: str, line_type1: str,
               color2: str, line_type2: str,
               mark_step: sp.core.numbers.Number,
               step_count: int,
               vertical_marks: Iterable[int]) -> str:
    x = np.linspace(float(left_border), float(right_border))
    y1 = func1(x)
    y2 = func2(x)
    fig, ax = plt.subplots()
    ax.set(xlim=[float(left_border), float(right_border)],
           ylim=[min(min(vertical_marks), y1.min(), y2.min()) - 1,
                 max(max(vertical_marks), y1.max(), y2.max()) + 1],
           xticks=[float(left_border) + i * float(mark_step) for i in range(step_count + 1)],
           xticklabels=
           [f"${latex(left_border + i * mark_step)}$" for i in range(step_count + 1)],
           yticks=vertical_marks)
    ax.plot(x, y1, color=color1, linestyle=line_type1)
    ax.plot(x, y2, color=color2, linestyle=line_type2)
    os.makedirs(os.path.dirname(OUT_ANSWER_PATH), exist_ok=True)
    fig.tight_layout()
    fig.savefig(OUT_ANSWER_PATH + str(option), bbox_inches='tight')
    return latex_image(OUT_ANSWER_PATH + str(option) + ".png")

In [29]:
@dataclass
class Color:
    human_version: str
    plot_version: str


@dataclass
class LineType:
    human_version: str
    plot_version: str

In [30]:
def generate_task_and_answer(option: int) -> Tuple[str, str]:
    def random_float_coef():
        return sp.S(choice(list(set(range(-5, 3)) - {0}))) / choice([1, 2, 3, 4, 6])

    left_border = random_float_coef()
    mark_step = abs(random_float_coef())
    step_count = randint(2, 4)
    right_border = left_border + step_count * mark_step
    borders_multiplier = choice([sp.pi, sp.E, 1, 1, 1, 1])
    left_border *= borders_multiplier
    right_border *= borders_multiplier
    mark_step *= borders_multiplier
    color1, color2 = choice([
        Color("зеленая", "green"),
        Color("синяя", "blue"),
        Color("черная", "black"),
        Color("желтая", "yellow"),
        Color("фиолетовая", "magenta")
    ], size=2, replace=False)
    line_type1, line_type2 = choice([
        LineType("линия из точек", "dotted"),
        LineType("пунктирная линия", "dashed"),
        LineType("сплошная линия", "solid"),
    ], size=2)

    bad_functions = True
    x = sp.Symbol('x')
    func1 = None
    func2 = None
    vertical_marks = None
    while bad_functions:
        func1, func2 = generate_func(), generate_func()

        x_linspace = np.linspace(float(left_border), float(right_border))
        func1_linspace = func1.numpy_version(x_linspace)
        func2_linspace = func2.numpy_version(x_linspace)

        func1_min, func1_max = func1_linspace.min(), func1_linspace.max()
        func2_min, func2_max = func2_linspace.min(), func2_linspace.max()

        if abs(func1_min - func2_min) > 120 or abs(func1_max - func2_max) > 120:
            continue

        if (func1_max - func1_min) / (func2_max - func2_min) > 3:
            continue

        try:
            vertical_marks = sorted(set([randint(min(int(func1_min), int(func2_min) + 1),
                                                 max(int(func1_max), int(func2_max) - 1)) for _ in range(5)]))
        except ValueError:
            continue

        if len(vertical_marks) < 3:
            continue

        if abs(max(vertical_marks)) > 100 or abs(min(vertical_marks)) > 100:
            continue
        bad_functions = False

    return (
        task_condition(func1.sympy_version(x),
                       func2.sympy_version(x),
                       left_border, right_border,
                       color1.human_version, line_type1.human_version,
                       color2.human_version, line_type2.human_version,
                       mark_step,
                       vertical_marks),
        solve_task(option,
                   func1.numpy_version,
                   func2.numpy_version,
                   left_border, right_border,
                   color1.plot_version, line_type1.plot_version,
                   color2.plot_version, line_type2.plot_version,
                   mark_step,
                   step_count,
                   vertical_marks)
    )

In [31]:
doc = DocumentGenerator(generate_task_and_answer, DESCRIPTION)

In [32]:
write_tasks_and_solutions(doc, "../out/tasks/task-4.tex", OUT_ANSWER_PATH + "answer-4.tex", 150)

C:\Users\User\AppData\Local\Temp\ipykernel_5080\1397299098.py:18: RuntimeWarning: overflow encountered in exp
  lambda arr: float(coef1) * numpy_version(float(coef2) * arr)
C:\Users\User\AppData\Local\Temp\ipykernel_5080\674450038.py:44: RuntimeWarning: divide by zero encountered in scalar divide
  if (func1_max - func1_min) / (func2_max - func2_min) > 3:
C:\Users\User\AppData\Local\Temp\ipykernel_5080\1397299098.py:48: RuntimeWarning: overflow encountered in power
  lambda arr: float(coef1) * float(coef2) ** arr
